In [ ]:
# IMPORTS
import pandas as pd
import numpy as np

## Reading separate csv files

In [ ]:
laps_df = pd.read_csv(r'laps.csv', encoding='utf-16')
fcyphases_df = pd.read_csv(r'fcyphases.csv', encoding='utf-16')
races_df = pd.read_csv(r'races.csv', encoding='utf-16')
drivers_df = pd.read_csv(r'drivers.csv', encoding='utf-16')

## Importing main.csv which contains the merged (laps, drivers, races) tables

In [ ]:
main_df = pd.read_csv(r'main.csv', encoding='utf-8')
dtypes = {"id": float, 
          "carno": float, 
          "initials": object, 
          "name": object,
           "race_id": float,
         "driver_id": float,
         "team": object,
         "teamcolor": object ,
         "enginemanufacturer": object,
          "gridposition": float,
          "status": object,
          "resultposition": float,
          "completedlaps": float,
          "speedtrap": float,
          "date": object,
          "season": object,
          "location": str,
          "availablecompounds": object,
          "comment" : object,
          "nolaps": float,
          "nolapsplanned": float,
          "tracklength": float,
          "position": float,
          "q1laptime": float,
          "q2laptime": float,
          "q3laptime": float,
          "lapno": float,
          "laptime": float,
          "racetime": float,
          "gap": float,
          "interval": float,
          "compound": object,
          "pitintime": object,
          "pitstopduration": object,
          "nextcompound": object,
          "startlapprog_vsc" : float ,
          "endlapprog_vsc": float,
          "age_vsc": float,
          "startlapprog_sc": float,
          "endlapprog_sc": float,
          "age_sc": float,
          "accidents": float,
          "failures": float,
          "startracetime": float,
          "endracetime": float,
          "startraceprog": float,
          "endraceprog": float,
          "startlap": float,
          "endlap": float,
          "type": object 
          
         }
main_df.rename(columns={'id_x': 'id'}, inplace=True)

In [ ]:
main_df = main_df.drop(['comment'], axis=1)

## Data Filtering

In [ ]:
# FILTER 1 - Removing WET races

# apply a filter to remove any rows where the "compound" column contains "W" or "I"
main_df = main_df[~main_df['compound'].isin(['W', 'I'])]

main_df

In [ ]:
# FILTER 2 - Removing data related to drivers making more than 3 pit stops

# Iterate over each unique combination of id and race_id
for id, race_id in main_df[['id', 'race_id']].drop_duplicates().values:
    # Get the rows for the current combination of id and race_id
    rows = main_df[(main_df['id'] == id) & (main_df['race_id'] == race_id)]

    # Check if the number of pit stops exceeds three
    if rows['pitstopduration'].count() > 3:
        # Remove the rows
        main_df = main_df.drop(rows.index)

In [ ]:
# FILTER 3 - Data relating to drivers with a result position greater than 15 are removed.

# create a new column called "result_position" and set the default value to NaN
main_df['result_position'] = float('nan')

# loop through each race in the database
for race_id in main_df['race_id'].unique():
    # get the last lap of the race
    last_lap = main_df[main_df['race_id'] == race_id]['lapno'].max()
    # loop through each driver in the race
    for driver_id in main_df[main_df['race_id'] == race_id]['id'].unique():
        # get the position of the driver on the last lap
        driver_position = main_df[(main_df['race_id'] == race_id) & (main_df['id'] == driver_id) & (main_df['lapno'] == last_lap)]['position'].values
        # if the driver completed the last lap, set their result position to their position on the last lap
        if len(driver_position) > 0:
            main_df.loc[(main_df['race_id'] == race_id) & (main_df['id'] == driver_id), 'result_position'] = driver_position[0]

# apply a filter to remove any rows where the result position is greater than 15
main_df = main_df[main_df['result_position'] < 15]

## Feature Engineering

In [ ]:
 def race_progress(main_df):
        
    r_id = main_df.race_id.unique()
    d_id = main_df.driver_id.unique()
    
    # create new df
    new_df = pd.DataFrame()

    for r in r_id:
        
        # rows of the same race
        x = main_df[main_df['race_id'] == r]
        
        for d in d_id:
            # rows of the same race and driver
            y = x[x['driver_id'] == d]
            
            if not y.empty:
                max_value = max(y['racetime'])
                y['race_progress'] = y['racetime'] / max_value
            
                new_df = pd.concat([new_df, y])
    
    return new_df

main_df = race_progress(main_df)

In [ ]:
# REMAINING PIT STOPS - CURRENT LAP

main_df['remaining_pit_stops'] = 3

for index, row in main_df.iterrows():
    if pd.notnull(row['pitintime']):
        driver_id = row['id']
        remaining_pit_stops = main_df.loc[(main_df['id'] == driver_id) & (main_df['lapno'] > row['lapno']), 'remaining_pit_stops'].max()
        main_df.loc[index, 'remaining_pit_stops'] = max(remaining_pit_stops - 1, 0)

In [ ]:
# RELATIVE COMPOUND - CURRENT LAP 

# Define a dictionary to map compound codes to tire types based on the given information
compound_dict = {
    "A1": {2014: "hard", 2015: "hard", 2016: "hard", 2017: "hard", 2018: "hard"},
    "A2": {2014: "medium", 2015: "medium", 2016: "medium", 2017: "medium", 2018: "hard", 2019: "hard"},
    "A3": {2014: "soft", 2015: "soft", 2016: "soft", 2017: "soft", 2018: "medium", 2019: "hard"},
    "A4": {2014: "soft", 2015: "soft", 2016: "soft", 2017: "soft", 2018: "soft", 2019: "medium"},
    "A5": {2016: "soft", 2017: "soft", 2018: "soft"},
    "A6": {2018: "soft", 2019: "soft"},
    "A7": {2018: "soft", 2019: "soft"}
}

# Define a function to map the compound codes to tire types based on the season
def map_compound(row):
    compound = row['compound']
    season = row['season']
    lapno = row['lapno']  # Get the current lap number
    
    if compound in compound_dict and season in compound_dict[compound]:
        return compound_dict[compound][season]
    elif lapno > 1:
        previous_lap_compound = main_df.at[row.name - 1, 'relativecompound']  # Get the relativecompound value from the previous lap
        return previous_lap_compound
    else:
        return None

# Apply the mapping function to the compound column to create the relativecompound column
main_df['relativecompound'] = main_df.apply(map_compound, axis=1)

In [ ]:
# Race Track Feature - aka location

In [ ]:
# Fulfilled Second Compound Feature

main_df_compounds = main_df.groupby(['driver_id', 'compound'])['lapno'].nunique().reset_index()
main_df['fulfilled_second_compound'] = main_df.apply(lambda row: 
    main_df_compounds[(main_df_compounds['driver_id'] == row['driver_id']) & 
                 (main_df_compounds['compound'] != row['compound'])]['lapno'].sum() >= 2,
    axis=1
)

In [ ]:
# Number of available compounds Feature

# create a new feature called 'number of available compounds'
main_df['number_of_available_compounds'] = np.where(main_df['season'] <= 2016, 2, 3)

In [ ]:
# Handling new NaN values
main_df.fillna(0, inplace=True)
display(main_df)

In [ ]:
main_df.to_csv(r'f1dataset2.csv', index=False)